# SQLite 데이터 관리

SQLite는 서버가 필요 없는 경량 데이터베이스로, 파일 기반으로 동작하여 간단한 데이터 저장 및 관리에 적합합니다.

## 파이썬에서 SQLite 사용하기

파이썬에서는 `sqlite3` 모듈을 사용하여 SQLite 데이터베이스를 다룰 수 있습니다. 이 모듈은 파이썬 표준 라이브러리에 포함되어 있어 별도의 설치가 필요 없습니다.

### 기본 사용 방법

1. **데이터베이스 연결**: `sqlite3.connect()`로 데이터베이스 파일에 연결
2. **커서 생성**: `cursor()` 메서드로 SQL 명령을 실행할 커서 생성
3. **SQL 실행**: `execute()` 메서드로 SQL 쿼리 실행
4. **변경사항 저장**: `commit()`으로 트랜잭션 커밋
5. **연결 종료**: `close()`로 데이터베이스 연결 종료

## 통신 요금제 데이터 관리

### 기본 세팅 - 테이블 생성

SQLite는 FOREIGN KEY가 기본 비활성화 상태로 DB 연결 시 FK 사용 활성활 필요

```python
PRAGMA foreign_keys = ON;
```

In [ ]:
import sqlite3

DB_FILE = "mno-data.db"

def get_connection():
    conn = sqlite3.connect(DB_FILE)
    conn.execute("PRAGMA foreign_keys = ON")
    return conn

def init_db():
    conn = get_connection()
    cur = conn.cursor()

    # 요금제 테이블
    cur.execute("""
    CREATE TABLE IF NOT EXISTS plans (
        plan_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        monthly_fee INTEGER NOT NULL,
        data_gb REAL NOT NULL,
        voice_minutes INTEGER NOT NULL,
        sms_count INTEGER NOT NULL,
        is_unlimited_data BOOLEAN NOT NULL DEFAULT 0,
        created_at TEXT DEFAULT (datetime('now'))
    );
    """)

    # 요금제 혜택 테이블
    cur.execute("""
    CREATE TABLE IF NOT EXISTS plan_benefits (
        benefit_id INTEGER PRIMARY KEY AUTOINCREMENT,
        plan_id INTEGER NOT NULL,
        benefit_name TEXT NOT NULL,
        benefit_desc TEXT,
        priority INTEGER,
        FOREIGN KEY (plan_id) REFERENCES plans (plan_id)
    );
    """)

    conn.commit()
    conn.close()

### 요금제 및 요금제 혜택 등록

In [ ]:
def create_plan(name, monthly_fee, data_gb, voice_minutes, sms_count, is_unlimited_data=False):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("""
        INSERT INTO plans (name, monthly_fee, data_gb, voice_minutes, sms_count, is_unlimited_data)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (name, monthly_fee, data_gb, voice_minutes, sms_count, int(is_unlimited_data)))

    conn.commit()
    plan_id = cur.lastrowid
    conn.close()
    return plan_id


def create_plan_benefit(plan_id, benefit_name, benefit_desc=None, priority=None):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("""
        INSERT INTO plan_benefits (plan_id, benefit_name, benefit_desc, priority)
        VALUES (?, ?, ?, ?)
    """, (plan_id, benefit_name, benefit_desc, priority))

    conn.commit()
    benefit_id = cur.lastrowid
    conn.close()
    return benefit_id

### 요금제 조회 (단건/전체/조인)

In [ ]:
def get_plan(plan_id):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("SELECT * FROM plans WHERE plan_id = ?", (plan_id,))
    row = cur.fetchone()

    conn.close()
    return row


def get_all_plans():
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("SELECT * FROM plans ORDER BY plan_id")
    rows = cur.fetchall()

    conn.close()
    return rows


def get_plan_with_benefits(plan_id):
    """요금제 + 해당 요금제의 혜택 목록을 함께 조회"""
    conn = get_connection()
    cur = conn.cursor()

    # 요금제 기본정보
    cur.execute("SELECT * FROM plans WHERE plan_id = ?", (plan_id,))
    plan = cur.fetchone()

    # 혜택 목록
    cur.execute("""
        SELECT benefit_id, benefit_name, benefit_desc, priority
        FROM plan_benefits
        WHERE plan_id = ?
        ORDER BY COALESCE(priority, 9999)
    """, (plan_id,))
    benefits = cur.fetchall()

    conn.close()
    return plan, benefits


### 요금제 정보 수정

In [ ]:
def update_plan_fee(plan_id, new_monthly_fee):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("""
        UPDATE plans
        SET monthly_fee = ?
        WHERE plan_id = ?
    """, (new_monthly_fee, plan_id))

    conn.commit()
    changed = cur.rowcount  # 변경된 row 수
    conn.close()
    return changed > 0


### 요금제 정보 삭제 - FK 순서 주의

In [ ]:
def delete_plan(plan_id):
    """요금제 삭제 전에 해당 요금제의 혜택도 같이 삭제(ON DELETE CASCADE가 없다면 직접 삭제)"""
    conn = get_connection()
    cur = conn.cursor()

    # 1) 혜택 먼저 삭제
    cur.execute("DELETE FROM plan_benefits WHERE plan_id = ?", (plan_id,))
    # 2) 요금제 삭제
    cur.execute("DELETE FROM plans WHERE plan_id = ?", (plan_id,))

    conn.commit()
    deleted = cur.rowcount
    conn.close()
    return deleted > 0


def delete_plan_benefit(benefit_id):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute("DELETE FROM plan_benefits WHERE benefit_id = ?", (benefit_id,))

    conn.commit()
    deleted = cur.rowcount
    conn.close()
    return deleted > 0


### 테스트 실행 예시

In [ ]:
init_db()

# 요금제 생성
plan_id = create_plan("T플랜 맥스", 100_000, 100.0, 2000, 1000, True)

# 요금제 혜택 추가
create_plan_benefit(plan_id, "Wavve 앤 데이터", "동영상 스트리밍 무제한", 1)
create_plan_benefit(plan_id, "FLO 앤 데이터", "음악 스트리밍 무제한", 2)

print("== 전체 요금제 ==")
for row in get_all_plans():
    print(row)

In [ ]:
# 특정 요금제 + 혜택 조회
plan, benefits = get_plan_with_benefits(plan_id)
print("PLAN:", plan)
print("BENEFITS:")
for b in benefits:
    print("  ", b)

In [ ]:
# 요금제 요금 수정
update_plan_fee(plan_id, 55000)
print("수정 후 요금제:", get_plan(plan_id))

In [ ]:
# 삭제 테스트
delete_plan(5)

# 삭제 후 전체 요금제 조회
for row in get_all_plans():
    print(row)